In [1]:
#XGBoost
# https://stackabuse.com/gradient-boosting-classifiers-in-python-with-scikit-learn/
# https://blogs.sas.com/content/subconsciousmusings/2017/04/12/machine-learning-algorithm-use/
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.externals import joblib
from sklearn.preprocessing import OneHotEncoder

C:\Users\Rybak\.conda\envs\Scraper3.6\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


# Data Selection from DB

In [2]:
#cnx_rent = sqlite3.connect('real-estate-rent.db')
cnx_buy = sqlite3.connect('real-estate.db')

#df_rent = pd.read_sql_query("SELECT * FROM immoscout", cnx_rent)
df_buy = pd.read_sql_query("SELECT * FROM immoscout", cnx_buy)
#cnx_rent.close()
cnx_buy.close()
#print('Rent Shape: ', df_rent.shape)
print('Buy Shape: ', df_buy.shape)
#df_rent.head()
df_buy.head()

Buy Shape:  (80, 39)


,immo_id,year,month,day,title,url,city,postcode,quarter,street,...,equipment,provision,provision_percent,land_transfer,land_transfer_percent,notary,notary_percent,entry_land,entry_land_percent,rent
0,118357961,2020,5,31,Top-Lage zum Schnäppchenpreis: Sanierungsbedür...,https://www.immobilienscout24.de/expose/118357961,Dortmund,44339,Brechten,,...,- Sanierungsbedürftige 2-Zimmer-Wohnung\n- Bal...,3177.30,3.57,5785.0,6.5,1335.0,1.5,445.0,0.5,0.0
1,117810775,2020,5,31,Erdgeschosswohnung mit 2 Garagen,https://www.immobilienscout24.de/expose/117810775,Dortmund,44329,Lanstrop,Hafnerstr.,...,None,7092.40,4.76,9685.0,6.5,2235.0,1.5,745.0,0.5,0.0
2,117052263,2020,5,31,++Eigentumswohnung in sehr ruhiger und gepfleg...,https://www.immobilienscout24.de/expose/117052263,Dortmund,44265,Niederhofen,Godefriedstr.,...,Die vermietete Wohnung ist hell und freundlich...,3803.24,4.76,5193.5,6.5,1198.5,1.5,399.5,0.5,0.0
3,118599037,2020,5,31,Gesucht. Gefunden. Gekauft! Machen Sie es sich...,https://www.immobilienscout24.de/expose/118599037,Dortmund,44388,Lütgendortmund,,...,None,5664.40,4.76,7735.0,6.5,1785.0,1.5,595.0,0.5,0.0
4,116644933,2020,5,31,Für Kapitalanleger oder Selbstnutzer: Eigent...,https://www.immobilienscout24.de/expose/116644933,Dortmund,44309,Neuasseln,,...,"Ölzentralheizung, Warmwasserbereitung über Ele...",2820.30,3.57,5135.0,6.5,1185.0,1.5,395.0,0.5,0.0


# Methods

In [3]:
def encode_data_for_training(df):
    # Prepare the training data for training
    # Features:
    # Data One Hot Encoding
    # will save the encoder files in the folder
    # use bag of words for one hot encoding
    
    #df.immo_id = df.immo_id.astype(str)
    
    #set to True or False and then to 1 or 0
    df["balcony"] = df["balcony"] == "true"
    df["balcony"] = df.balcony.astype(int)
    df["cellar"] = df["cellar"] == "true"
    df["cellar"] = df.cellar.astype(int)
    df["garden"] = df["garden"] == "true"
    df["garden"] = df.garden.astype(int)
    df["lift"] = df["lift"] == "true"
    df["lift"] = df.lift.astype(int)
    
    # one hot city encoding
    enc_city = OneHotEncoder(handle_unknown='ignore')
    enc_city.fit(df[['city']])
    df_city_enc = pd.DataFrame(data=enc_city.transform(df[['city']]).toarray(), columns=enc_city.get_feature_names(['city']), dtype=bool)
    df_city_enc = df_city_enc * 1
    # save 
    joblib.dump(enc_city, 'encoder_city.joblib')

    # one hot quarter encoding
    enc_quarter = OneHotEncoder(handle_unknown='ignore')
    enc_quarter.fit(df2[['quarter']])
    df_quarter_enc = pd.DataFrame(data=enc_quarter.transform(df[['quarter']]).toarray(), columns=enc_quarter.get_feature_names(['quarter']), dtype=bool)
    df_quarter_enc = df_quarter_enc * 1
    # save 
    joblib.dump(enc_quarter, 'encoder_quarter.joblib')

    # concatenate training data
    X = pd.concat((df[['balcony', 'cellar', 'garden', 'lift', 'livingSpace', 'numberOfRooms']], df_city_enc, df_quarter_enc), axis=1)
    y = df[["value"]]
    X = bag_of_words(X, df)
    return (X, y)

def predict_data(df, model, enc_city, enc_quarter):
    # predict data with given model
    # the formating of the data should be the same as the trained model
    
    #set to True or False and then to 1 or 0
    df["balcony"] = df["balcony"] == "true"
    df["balcony"] = df.balcony.astype(int)
    df["cellar"] = df["cellar"] == "true"
    df["cellar"] = df.cellar.astype(int)
    df["garden"] = df["garden"] == "true"
    df["garden"] = df.garden.astype(int)
    df["lift"] = df["lift"] == "true"
    df["lift"] = df.lift.astype(int)
    
    # enc_city, enc_quarter
    hot_city = pd.DataFrame(data=enc_city.transform(df[['city']]).toarray(), columns=enc_city.get_feature_names(['city']), dtype=bool)
    hot_city = hot_city * 1
    hot_quarter = pd.DataFrame(data=enc_quarter.transform(df[['quarter']]).toarray(), columns=enc_quarter.get_feature_names(['quarter']), dtype=bool)
    hot_quarter = hot_quarter * 1
    # data for prediction
    X = pd.concat((df[['balcony', 'cellar', 'garden', 'lift', 'livingSpace', 'numberOfRooms']], hot_city, hot_quarter), axis=1)
    y = df[["value"]]
    X = bag_of_words(X, df)
    y_pred = model.predict(X)
    return y_pred
    
def bag_of_words(X, df):
    # BagOfWords implementation
    # As no nice and easy implementation was found this was the next best thing.
    # For every word in the vacabulary a column is crated and added if the text includes the corresponding word.
    vocabulary = [ "uni", "modern", "dach", "loft", "pool", "wg", "altbau", "luxu", "terasse", 
                  "neubau", "maisonet", "penthouse", "erstbezug", "kamin", "langzeit", "renoviert", "dachgeschoss"]
    for word in vocabulary:
        X[word] = 0
        for index, row in df.iterrows():
            if word.lower() in row["title"].lower():
                #X.set_value(index, word, 1)
                X.at[index, word] = 1
    return X

# Buy Analysis
## load Model

In [4]:
filename = 'model_1_max.sav'
loaded_model = joblib.load(filename)
loaded_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=nan, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

## Predict Data

In [5]:
%%capture
df2 = df_buy[["title", "city", "quarter", "balcony", "cellar",
         "garden", "lift", "livingSpace", "numberOfRooms", "value"]]

encoder_city = joblib.load('encoder_city.joblib')
encoder_quarter = joblib.load('encoder_quarter.joblib')

y_pred = predict_data(df2, loaded_model, encoder_city, encoder_quarter)

In [6]:
y_pred.shape

(80,)

# Data Filtering and Calculations

In [7]:
#selection testing
#df_output[df_output.realtorCompanyName == "HAK Immobilien GmbH"]
#df_output['year_of_construction']
#df_output[df_output.year_of_construction == "unbekannt"]

Dana Filtering

In [8]:
df_output = df_buy.copy(deep=False)
df_output['prediction'] = y_pred
#df_output['all_costs'] = df_buy['provision'] + df_buy['land_transfer'] + df_buy['notary'] + df_buy['entry_land']+ df_buy['value']
#df_output['rendite_netto'] = ( 12 * df_output['prediction'] ) / df_buy['value']
#df_output['rendite_brutto'] = ( 12 * df_output['prediction'] ) / df_buy['all_costs']
#df_output['preis_m2'] = df_buy['value'] / df_buy['livingSpace']
df_output.drop(columns=['object_description', 'area', 'additional_info', 'equipment'], inplace = True)
# delete HAK
df_output.drop(df_output[df_output.realtorCompanyName == "HAK Immobilien GmbH"].index, inplace=True)
# no too old
#df_output.astype({'year_of_construction': 'int32'})
#df_output.drop(df_output[df_output.year_of_construction < 1960].index, inplace=True)
df_output.head(100)

,immo_id,year,month,day,title,url,city,postcode,quarter,street,...,provision,provision_percent,land_transfer,land_transfer_percent,notary,notary_percent,entry_land,entry_land_percent,rent,prediction
0,118357961,2020,5,31,Top-Lage zum Schnäppchenpreis: Sanierungsbedür...,https://www.immobilienscout24.de/expose/118357961,Dortmund,44339,Brechten,,...,3177.30,3.57,5785.0,6.5,1335.0,1.5,445.0,0.5,0.0,456.116669
1,117810775,2020,5,31,Erdgeschosswohnung mit 2 Garagen,https://www.immobilienscout24.de/expose/117810775,Dortmund,44329,Lanstrop,Hafnerstr.,...,7092.40,4.76,9685.0,6.5,2235.0,1.5,745.0,0.5,0.0,549.367676
2,117052263,2020,5,31,++Eigentumswohnung in sehr ruhiger und gepfleg...,https://www.immobilienscout24.de/expose/117052263,Dortmund,44265,Niederhofen,Godefriedstr.,...,3803.24,4.76,5193.5,6.5,1198.5,1.5,399.5,0.5,0.0,410.524689
3,118599037,2020,5,31,Gesucht. Gefunden. Gekauft! Machen Sie es sich...,https://www.immobilienscout24.de/expose/118599037,Dortmund,44388,Lütgendortmund,,...,5664.40,4.76,7735.0,6.5,1785.0,1.5,595.0,0.5,0.0,656.764465
4,116644933,2020,5,31,Für Kapitalanleger oder Selbstnutzer: Eigent...,https://www.immobilienscout24.de/expose/116644933,Dortmund,44309,Neuasseln,,...,2820.30,3.57,5135.0,6.5,1185.0,1.5,395.0,0.5,0.0,380.788910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,111349605,2020,5,31,*o. zzgl. Käufercourtage* Wohnträume am Park: ...,https://www.immobilienscout24.de/expose/111349605,Dortmund,44225,Renninghausen,Am Bahnhof Tierpark,...,0.00,0.00,7735.0,6.5,1785.0,1.5,595.0,0.5,0.0,489.302643
74,114172552,2020,5,31,Attraktive 2 Zimmer Wohnung in Dortmund Eving ...,https://www.immobilienscout24.de/expose/114172552,Dortmund,44339,Eving,,...,4250.00,5.00,5525.0,6.5,1275.0,1.5,425.0,0.5,0.0,386.135590
75,114173746,2020,5,31,Attraktive 2 Zimmer Wohnung in Dortmund Eving ...,https://www.immobilienscout24.de/expose/114173746,Dortmund,44339,Eving,,...,4250.00,5.00,5525.0,6.5,1275.0,1.5,425.0,0.5,0.0,386.135590
76,63827130,2020,5,31,Schöne helle und großzügige 2 Zimmer Wohnung m...,https://www.immobilienscout24.de/expose/63827130,Dortmund,44145,Innenstadt,Brunnenstrasse,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,410.524689


# Save to CSV

In [9]:
df_output.to_csv("Data/data_predict.csv", sep=';', decimal=",", encoding='utf-8-sig')